<a href="https://colab.research.google.com/github/KorStats/skku/blob/main/code/Back_transalation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 데이터 불러오기

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
ln: failed to create symbolic link '/mydrive/My Drive': File exists
'Colab Notebooks'   법률_train_original.csv	  신문기사_valid_original.csv
 kosbi_train.json   법률_valid_original.csv	  이전
'My Drive'	    신문기사_train_original.csv   졸업논문


In [3]:
import pandas as pd
df=pd.read_csv('/content/gdrive/MyDrive/졸업논문/DATA/예시데이터.csv', encoding='utf-8-sig')

In [4]:
df.columns

Index(['demographic_category', 'demographic_group', 'context', 'context_en',
       'sentence', 'sentence_en', 'context_label', 'context_sub_label',
       'sentence_label', 'sentence_sub_label', 'annotated_demo'],
      dtype='object')

In [5]:
df['text']=df['context']+df['sentence']

In [6]:
df_1=df[df['sentence_label']=='safe']
df_2=df[df['sentence_label']=='unsafe']

# Back-translation

In [7]:
pip install googletrans==4.0.0-rc1

In [65]:
import pandas as pd
from googletrans import Translator

def back_translate(text, source_lang, pivot_lang):
    translator = Translator()
    translated_to_pivot = translator.translate(text, src=source_lang, dest=pivot_lang).text
    back_translated = translator.translate(translated_to_pivot, src=pivot_lang, dest=source_lang).text
    return back_translated

def augment_data(texts, source_lang, pivot_langs, multiplier):
    augmented_texts = []
    for text in texts:
        for _ in range(multiplier - 1):
            for pivot_lang in pivot_langs:
                augmented_texts.append(back_translate(text, source_lang, pivot_lang))
    return augmented_texts

def augment_dataframe(df, text_column, category_column, target_category, source_lang, pivot_langs, multiplier):
    df_augmented = df.copy()
    target_texts = df[df[category_column] == target_category][text_column].tolist()
    augmented_texts = augment_data(target_texts, source_lang, pivot_langs, multiplier)
    augmented_rows = [{text_column: text, category_column: target_category} for text in augmented_texts]
    df_augmented = pd.concat([df_augmented, pd.DataFrame(augmented_rows)], ignore_index=True)
    return df_augmented

# 증분하기

###### df_2를 증분

In [66]:
print(df_1.shape)
print(df_2.shape)

(3, 12)
(4, 12)


In [67]:
df_2.columns

Index(['demographic_category', 'demographic_group', 'context', 'context_en',
       'sentence', 'sentence_en', 'context_label', 'context_sub_label',
       'sentence_label', 'sentence_sub_label', 'annotated_demo', 'text'],
      dtype='object')

In [75]:
df=df_2
#'en', 'ja', 'zh-cn', 'zh-tw', 'fr', 'de', 'es', 'ru', 'pt', 'hi'
#원데이터 + multiplier-1 * pivot_languages 수
pivot_languages = ['en']
multiplier=2
df_2_augmented = augment_dataframe(df, text_column='text', category_column='sentence_label', target_category='unsafe', source_lang='ko', pivot_langs=pivot_languages, multiplier=multiplier)

In [76]:
df_2_augmented.shape

(8, 12)

# 증분데이터 저장하기

In [77]:
df_aug=pd.concat([df_1, df_2_augmented])

In [78]:
df_aug['sentence_label'].value_counts()

,count
sentence_label,
unsafe,8
safe,3


In [ ]:
df_aug.to_csv('증분데이터.csv', encoding='utf-8-sig')